In [8]:
import os
from rich.console import Console
from rich.panel import Panel
from rich.markdown import Markdown
from rich import print as rprint

from openai import OpenAI

from settings import OpenAISettings

import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

console = Console()


def get_client() -> OpenAI:
    settings = OpenAISettings()
    client = OpenAI(
        api_key=settings.openai_api_key.get_secret_value(),
        base_url=str(settings.openai_base_url),
    )
    return client


def responses_variant():
    client = get_client()

    completion = client.responses.create(
        model="Qwen/Qwen3-Next-80B-A3B-Instruct",
        instructions="Ты русский православный батюшка матершинник, который составляет молитвы людям с кучей мата и эмодзи",
        input="Составь молитву, чтобы не ломался автобус от ДГТУ до Шаповалова",
    )

    rprint(completion.output_text)


messages = [
    {
        "role": "system",
        "content": "Ты русский православный батюшка матершинник, который составляет молитвы людям с кучей мата и эмодзи",
    },
]


def completions_variant():
    client = get_client()
    
    console.print(Panel.fit(
        "[bold cyan]Дарова[/bold cyan]\n\n"
        "Команды:\n"
        "  [yellow]/exit[/yellow] - выйти с зоны\n"
        "  [yellow]/clear[/yellow] - почистить туалет\n"
        "  [yellow]/system <текст>[/yellow] - замолвить словечко за лишний базар, шоб нормально общался",
        title="Тупая машина",
        border_style="white"
    ))
    
    while True:
        try:
            console.print("\n[bold green]Вы:[/bold green] ", end="")
            user_input = input()
            
            if not user_input.strip():
                continue
            
            if user_input.strip() == "/exit":
                console.print(Panel(
                    "[bold red1]Гудбай[/bold red1]",
                    border_style="yellow"
                ))
                break
            
            if user_input.strip() == "/clear":
                system_prompt = messages[0]
                messages.clear()
                messages.append(system_prompt)
                console.print(Panel(
                    "[bold yellow]История очищена! 🗑️[/bold yellow]",
                    border_style="yellow"
                ))
                continue
            
            if user_input.strip().startswith("/system "):
                new_system_prompt = user_input[8:].strip()
                if new_system_prompt:
                    messages[0] = {
                        "role": "system",
                        "content": new_system_prompt,
                    }
                    console.print(Panel(
                        f"[bold yellow]По другому буду базарить, сорян[/bold yellow]\n\n{new_system_prompt}",
                        border_style="yellow"
                    ))
                else:
                    console.print(Panel(
                        "[bold red]Ты че сволочь, а ну сказала шо нить что базарить, чо ты такой скромный а[/bold red]",
                        border_style="red"
                    ))
                continue
            
            messages.append({
                "role": "user",
                "content": user_input,
            })
            
            completion = client.chat.completions.create(
                model="Qwen/Qwen3-Next-80B-A3B-Instruct",
                messages=messages,
            )
            
            assistant_response = completion.choices[0].message.content
            
            messages.append({
                "role": "assistant",
                "content": assistant_response,
            })
            
            markdown_response = Markdown(assistant_response)
            console.print(Panel(
                markdown_response,
                title="[bold cyan]Тупая машина[/bold cyan]",
                border_style="green"
            ))
            
        except KeyboardInterrupt:
            console.print("\n")
            console.print(Panel(
                "[bold yellow]Гудбай[/bold yellow]",
                border_style="yellow"
            ))
            break
        except Exception as e:
            console.print(Panel(
                f"[bold red]Ошибка: {str(e)}[/bold red]",
                border_style="red"
            ))


if __name__ == "__main__":
    completions_variant()


╭──────────────────────────────── Тупая машина ─────────────────────────────────╮
│ Дарова                                                                        │
│                                                                               │
│ Команды:                                                                      │
│   /exit - выйти с зоны                                                        │
│   /clear - почистить туалет                                                   │
│   /system <текст> - замолвить словечко за лишний базар, шоб нормально общался │
╰───────────────────────────────────────────────────────────────────────────────╯

Вы:

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Гудбай                                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯